<a href="https://colab.research.google.com/github/QuinteroR/CuTonala_2024_A/blob/RaulQM/CountingSort3Codes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install mpi4py
!pip install numba

# CountSort

In [1]:
import time

start_time = time.time() #inicia el conteo de tiempo

def counting_sort(arr):
    # Encontrar el rango máximo en el arreglo
    max_value = max(arr)

    # Crear un arreglo de conteo para contar las ocurrencias de cada elemento
    count = [0] * (max_value + 1)

    # Contar las ocurrencias de cada elemento en el arreglo original
    for num in arr:
        count[num] += 1

    # Reconstruir el arreglo ordenado utilizando el arreglo de conteo
    sorted_arr = []
    for i in range(max_value + 1):
        if count[i] > 0:
            sorted_arr.extend([i] * count[i])

    return sorted_arr

# Ejemplo de uso
arr = [4, 2, 2, 8, 3, 3, 1, 5, 6, 7, 3, 4, 5, 2, 3, 4, 6, 5, 12, 2, 3, 3, 2, 1, 5, 7, 8, 6, 7, 9, 5, 6, 3, 7, 5, 9, 10, 11, 24, 35, 14, 55]
print("Arreglo original:", arr)

sorted_arr = counting_sort(arr)
print("Arreglo ordenado:", sorted_arr)

time.sleep(1)
end_time = time.time()
execution_time = end_time - start_time
print("tiempo de ejecución = ", execution_time)

Arreglo original: [4, 2, 2, 8, 3, 3, 1, 5, 6, 7, 3, 4, 5, 2, 3, 4, 6, 5, 12, 2, 3, 3, 2, 1, 5, 7, 8, 6, 7, 9, 5, 6, 3, 7, 5, 9, 10, 11, 24, 35, 14, 55]
Arreglo ordenado: [1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 9, 9, 10, 11, 12, 14, 24, 35, 55]
tiempo de ejecución =  1.0056161880493164


# CountSort Paralelizado con Mpi4Py

In [7]:


from mpi4py import MPI
import numpy as np

start_time = MPI.Wtime()

def counting_sort_parallel(arr):
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    arr_np = np.array(arr, dtype=np.int32)
    max_value = max(arr_np)
    temp_arr = np.zeros_like(arr_np)

    # Realizar la dispersión del arreglo local
    comm.Scatter([arr_np, MPI.INT], [temp_arr, MPI.INT], root=0)

    # Determinar el tamaño real del arreglo local
    local_size = len(temp_arr)

    # Crear un arreglo de conteo para contar las ocurrencias de cada elemento
    count = np.zeros(max_value + 1, dtype=np.int32)

    # Contar las ocurrencias de cada elemento en el arreglo local
    for num in temp_arr:
        count[num] += 1

    # Recolectar los conteos de cada proceso
    global_count = np.zeros_like(count)
    comm.Allreduce(count, global_count, op=MPI.SUM)

    # Reconstruir el arreglo ordenado utilizando el arreglo de conteo
    sorted_arr = np.zeros(sum(global_count), dtype=np.int32)
    offset = np.zeros(max_value + 1, dtype=np.int32)

    for i in range(1, max_value + 1):
        offset[i] = offset[i - 1] + global_count[i - 1]

    for i in range(local_size):
        sorted_arr[offset[temp_arr[i]]] = temp_arr[i]
        offset[temp_arr[i]] += 1

    return sorted_arr.tolist()

# Ejemplo de uso
if __name__ == "__main__":
    arr = [4, 2, 2, 8, 3, 3, 1, 5, 6, 7, 3, 4, 5, 2, 3, 4, 6, 5, 12, 2, 3, 3, 2, 1, 5, 7, 8, 6, 7, 9, 5, 6, 3, 7, 5, 9, 10, 11, 24, 35, 14, 55]
    print("Arreglo original:", arr)

    sorted_arr = counting_sort_parallel(arr)
    print("Arreglo ordenado:", sorted_arr)

    end_time = MPI.Wtime()
    print(f"Tiempo de ejecución =  {end_time - start_time:.8f} segundos")

Arreglo original: [4, 2, 2, 8, 3, 3, 1, 5, 6, 7, 3, 4, 5, 2, 3, 4, 6, 5, 12, 2, 3, 3, 2, 1, 5, 7, 8, 6, 7, 9, 5, 6, 3, 7, 5, 9, 10, 11, 24, 35, 14, 55]
Arreglo ordenado: [1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 9, 9, 10, 11, 12, 14, 24, 35, 55]
Tiempo de ejecución =  0.00604732 segundos


# Countsort Paralelizado con Numba

In [10]:
# Importamos las librerias
import numpy as np
from numba import njit, prange

start_time = time.time() #inicia el conteo de tiempo

@njit(parallel=True) #Utilizamos los decoradores
def counting_sort_parallel(arr):
    max_value = np.max(arr)
    count = np.zeros(max_value + 1, dtype=np.int32)

    # Cuenta las apariciones de cada elemento en paralelo
    for num in prange(len(arr)):
        count[arr[num]] += 1

    sorted_arr = np.empty(len(arr), dtype=arr.dtype)
    index = 0

    # Reconstruimos la matriz ordenada en paralelo
    for i in prange(len(count)):
        for j in range(count[i]):
            sorted_arr[index] = i
            index += 1

    return sorted_arr

# ejemplificamos
arr = np.array([4, 2, 2, 8, 3, 3, 1, 5, 6, 7, 3, 4, 5, 2, 3, 4, 6, 5, 12, 2, 3, 3, 2, 1, 5, 7, 8, 6, 7, 9, 5, 6, 3, 7, 5, 9, 10, 11, 24, 35, 14, 55], dtype=np.int64)
print("Original array:", arr)

sorted_arr = counting_sort_parallel(arr)
print("Sorted array:", sorted_arr)

time.sleep(1)
end_time = time.time()
execution_time = end_time - start_time
print("tiempo de ejecución = ", execution_time)

Original array: [ 4  2  2  8  3  3  1  5  6  7  3  4  5  2  3  4  6  5 12  2  3  3  2  1
  5  7  8  6  7  9  5  6  3  7  5  9 10 11 24 35 14 55]
Sorted array: [ 1  1  2  2  2  2  2  3  3  3  3  3  3  3  4  4  4  5  5  5  5  5  5  6
  6  6  6  7  7  7  7  8  8  9  9 10 11 12 14 24 35 55]
tiempo de ejecución =  2.9646246433258057
